In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb
import HEI
import sys
import networkx as nx

In [2]:
basepath='/Users/gracer/Google Drive/BCP/data/derivived'

In [3]:
convers = pd.read_csv('/Users/gracer/Google Drive/BCP/data/conversions.csv', sep=',')

In [15]:
mom_interest=['Participant ID', 'Participant ID_x','Date of Intake','HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN','HEIX3_TOTALFRUIT','HEIX4_WHOLEFRUIT','HEIX5_WHOLEGRAIN','HEIX6_TOTALDAIRY','HEIX7_TOTPROT','HEIX8_SEAPLANT_PROT','HEIX11_REFINEDGRAIN','HEIX12_ADDEDSUGARS','HEIX13_SATFATS','HEIX9_FATTYACID','HEIX10_SODIUM','HEI2015_TOTAL_SCORE']
ped_interest=['Participant ID_x','Participant ID', 'Date of Intake', 'Age at Intake','HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEI2015_TOTAL_SCORE','HEIX2_TOTALFRUIT','HEIX3_WHOLEGRAIN','HEIX4_TOTALDAIRY','HEIX5_PROTEIN','HEIX6_REFINEDGRAIN','HEIX7_FRUITJUICE','HEIX8_SSB','HEIX9_SWEETS','HEIX10_SALTY']
infant_interest=['Participant ID_x','Participant ID', 'Date of Intake', 'Age at Intake','HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEI2015_TOTAL_SCORE','HEIX2_TOTALFRUIT','HEIX3_WHOLEGRAIN','HEIX4_TOTALDAIRY','HEIX5_PROTEIN','HEIX6_REFINEDGRAIN','HEIX7_FRUITJUICE','HEIX8_SSB','HEIX9_SWEETS','HEIX10_SALTY','HEIX11_CEREAL']

In [16]:
data_dict={'infant':{}, 'young':{}, 'child':{}, 'mom':{}}
for item in glob.glob(os.path.join(basepath,'*HEI.csv')):
    if 'infant' in item:
        key='infant'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[infant_interest]
        data_dict['infant']=tmp
    if 'young' in item:
        key='young'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[ped_interest]
        data_dict['young']=tmp
    if 'child' in item:
        key='child'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[ped_interest]
        data_dict['child']=tmp
    if 'Mom' in item:
        key='Mom'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[mom_interest]
        data_dict['mom']=tmp

pre child (558, 248)
post child (209, 248)
pre infant (157, 249)
post infant (127, 249)
pre Mom (112, 414)
post Mom (87, 414)
pre young (126, 248)
post young (95, 248)


In [17]:
data_dict['young']

,Participant ID_x,Participant ID,Date of Intake,Age at Intake,HEIX0_BREASTFEEDING,HEIX1_VEGETABLES,HEI2015_TOTAL_SCORE,HEIX2_TOTALFRUIT,HEIX3_WHOLEGRAIN,HEIX4_TOTALDAIRY,HEIX5_PROTEIN,HEIX6_REFINEDGRAIN,HEIX7_FRUITJUICE,HEIX8_SSB,HEIX9_SWEETS,HEIX10_SALTY
0,105040,1995,2017-06-08,9.527916,0,2.5,37.5,5.0,2.5,0,5,2.5,5.0,5.0,5.0,5.0
1,107008,648900,2017-06-15,9.199368,10,5.0,47.5,5.0,0.0,5,0,2.5,5.0,5.0,5.0,5.0
2,107008,648900,2017-06-24,9.495062,10,5.0,47.5,5.0,0.0,5,0,2.5,5.0,5.0,5.0,5.0
3,11228,156965,2018-02-13,11.959178,10,5.0,45.0,5.0,2.5,0,5,2.5,5.0,5.0,0.0,5.0
4,11228,156965,2017-11-27,9.396497,10,5.0,45.0,5.0,2.5,0,0,2.5,5.0,5.0,5.0,5.0
5,116056,119470,2017-06-15,9.462207,0,2.5,27.5,5.0,2.5,0,0,2.5,5.0,5.0,0.0,5.0
6,131015,317472,2016-04-06,11.039241,0,2.5,32.5,2.5,2.5,5,0,2.5,5.0,5.0,5.0,2.5
7,131015,317472,2016-01-20,8.509415,0,2.5,35.0,2.5,2.5,5,0,2.5,5.0,5.0,5.0,5.0
8,137012,886496,2017-05-27,10.185014,0,5.0,40.0,5.0,2.5,0,5,2.5,5.0,5.0,5.0,5.0
10,151307,251117,2018-03-21,9.560771,10,5.0,52.5,2.5,2.5,5,5,2.5,5.0,5.0,5.0,5.0


In [23]:
inf_young = list(set(data_dict['infant']['Participant ID_x']).intersection(data_dict['young']['Participant ID_x']))
len(inf_young)

34

In [24]:
young_child = list(set(data_dict['child']['Participant ID_x']).intersection(data_dict['young']['Participant ID_x']))
len(young_child)

41

In [25]:
inf2child = list(set(inf_young).intersection(young_child))
len(inf2child)

25

In [26]:
inf_child = list(set(data_dict['child']['Participant ID_x']).intersection(data_dict['infant']['Participant ID_x']))
len(inf_child)

41

In [28]:
inf_mom = list(set(data_dict['infant']['Participant ID']).intersection(data_dict['mom']['Participant ID']))
len(inf_mom)

32

In [29]:
young_mom = list(set(data_dict['young']['Participant ID']).intersection(data_dict['mom']['Participant ID']))
len(young_mom)

30

In [30]:
child_mom = list(set(data_dict['mom']['Participant ID']).intersection(data_dict['child']['Participant ID']))
len(child_mom)

35